In [2]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
import json
import codecs
from bs4 import BeautifulSoup
import os

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', '&quot', 'datum', 'work', 'connect', 'code', 'iot', 'make', 'add', 'message', 'follow', 'solution', 'day'])

datafile = json.load(codecs.open('data.json', 'r', 'utf-8-sig'))

data = []
for item in datafile["items"]:
    data.append(item["body"])
    
#Remove Emoji
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
    
data_notags = []
#Remove tags and break lines
for item in data:
    data_bl = item.replace('\n', ' ').replace('\r', '')
    data_nc = re.sub(r'<pre>.+?</pre>', '', data_bl)
    data_ne = deEmojify(data_nc)
    data_nt = BeautifulSoup(data_ne, "lxml").text

    data_notags.append(data_nt)

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data_notags))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=2, no_above=1.0)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = 'C:/Users/ThIaG/PycharmProjects/lda3/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, id2word=id2word)


# Show Topics
pprint(ldamallet.show_topics(num_topics = 12, formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\n\n\nGrau de Coerência: ', coherence_ldamallet, '\n\n')

def create_file():
    path = "files/topico"
    ext = ".json"
    for i in range(0,12):
        os.remove(path + str(i) + ext)
        open(path + str(i) + ext, "x")
        

def save_data(index, topic_num, data):
    json_object = json.dumps(data["items"][index])
    with open("files/topico"+ str(topic_num) + ".json", "a") as outfile:
        outfile.write(json_object)


def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data, alldata=datafile):
    # Init output
    sent_topics_df = pd.DataFrame()

    create_file()
    
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                save_data(i, topic_num, alldata)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data, alldata=datafile)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution = topic_contribution.rename_axis('Dominant_Topic').reset_index(name='percentage')

topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']].drop_duplicates()
topic_num_keywords.index = range(len(topic_num_keywords))

df_dominant_topics = pd.merge(topic_contribution, topic_num_keywords, how='inner', on='Dominant_Topic')
df_dominant_topics.head(12)

[(0,
  [('run', 0.06426276329882184),
   ('file', 0.06328097108175651),
   ('window', 0.04926811852909675),
   ('raspberry_pi', 0.03641556586933238),
   ('build', 0.034630489111031774),
   ('core', 0.02552659764369868),
   ('script', 0.021063905747947163),
   ('version', 0.02079614423420207),
   ('instal', 0.019011067475901464),
   ('project', 0.017672259907176007)]),
 (1,
  [('app', 0.09259617243379084),
   ('application', 0.08563696114440364),
   ('android', 0.04716798762806882),
   ('project', 0.04243185772279142),
   ('work', 0.02696694374637541),
   ('method', 0.02687028803402281),
   ('java', 0.02368064952638701),
   ('find', 0.02184419099168761),
   ('develop', 0.021070945292866808),
   ('run', 0.01875120819640441)]),
 (2,
  [('create', 0.06015602970203966),
   ('command', 0.052824513582103584),
   ('request', 0.052730519785694145),
   ('aw', 0.048406805150860044),
   ('update', 0.044741047090892),
   ('thing', 0.04436507190525425),
   ('api', 0.025754300216185733),
   ('type', 

,Dominant_Topic,percentage,Topic_Keywords
0,0.0,0.1133,"run, file, window, raspberry_pi, build, core, ..."
1,5.0,0.1127,"server, network, connect, internet, web, contr..."
2,8.0,0.0961,"sensor, esp, temperature, read, output, displa..."
3,9.0,0.0943,"datum, database, store, time, query, good, pro..."
4,3.0,0.0876,"client, message, topic, publish, connection, l..."
5,7.0,0.0815,"device, service, cloud, user, base, gateway, c..."
6,11.0,0.0815,"azure, hub, follow, show, create, sample, issu..."
7,4.0,0.0812,"send, datum, receive, event, data, platform, n..."
8,2.0,0.0765,"create, command, request, aw, update, thing, a..."
9,1.0,0.0625,"app, application, android, project, work, meth..."
